In [1]:
%pip install beautifulsoup4
%pip install requests


Note: you may need to restart the kernel to use updated packages.


In [40]:
# GLOBAL variable
# seed_url = ''
# first 5000 website from 'https://www.wongnai.com/', 'https://food.trueid.net/', 'https://www.ryoiireview.com/'
# after 5000 website from 'https://www.7eleven.co.th/','https://sou-dai.com/','https://numsups.com/'
frontier_q = ['https://www.7eleven.co.th/','https://sou-dai.com/','https://numsups.com/']
visited_q = []
visited_host = set()
# KU_DOMAIN = 'ku.ac.th'
KEY_WORD = ('วัตถุดิบ','แคลอรี่','อาหาร','เมนู','ของกิน','กับข้าว','รสชาติ','อร่อย','เครื่องเคียง','ของว่าง','เครื่องดื่ม','ขนม')
PAGE_COUNT = 5000
WEB_TYPE = ('html')
FILE_TYPE = ('pdf', 'doc', 'xls', 'ppt', 'exe', 'jpg', 'mpg', 'zip', 'xlsx', 'png')
headers = {
    'User-Agent': 'Atipat Pankong',
    'From': 'atipat.pa@ku.th'
}
debug=True

In [38]:
# check backup file
try:
    f = open('pending_queue.txt' ,'r')
    s = f.read()
    if(s):
        url_q = s.splitlines()
        PAGE_COUNT = int(url_q[0])
        frontier_q = url_q[1:]
    if(debug):
        print(PAGE_COUNT)
        print(frontier_q)
except Exception as err:
    print(err)
    print('no backup')
    PAGE_COUNT = 0
    # frontier_q = [seed_url]

'charmap' codec can't decode byte 0x89 in position 1064302: character maps to <undefined>
no backup


In [41]:
frontier_q

['https://www.7eleven.co.th/', 'https://sou-dai.com/', 'https://numsups.com/']

In [42]:

# print for debug function.
def show_debug(url,error=False):
    global PAGE_COUNT
    if(not error):
        PAGE_COUNT += 1
        print('page: ',PAGE_COUNT)
#     print('site: ',url)
#     print('-'*50)

In [43]:
# FIFO queue
def enqueue(links):
    global frontier_q
    for link in links:
        if link not in frontier_q and link not in visited_q and not link.endswith(FILE_TYPE) and not 'mailto:' in link:
            # print('in')
            frontier_q.append(link)
        # else : 
        #     print('out')

def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

In [44]:
import requests
from requests.exceptions import HTTPError
from urllib.parse import urljoin, urlparse

def get_page(url,require_content_type='text/html',join_set=True):
    global headers
    text = ''
    try:
        if(url.endswith(FILE_TYPE) or 'mailto:' in url): return text
        response = requests.get(url, headers=headers, timeout=2)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    else:
        if(require_content_type in response.headers['Content-Type']):
            # response.encoding = response.apparent_encoding
            text = response.text
            if(sum([1 for x in KEY_WORD if x in text])<3):
                return ""
            if 'facebook' in url or 'youtube' in url or 'google' in url or 'instagram' in url or 'twitter' in url or 'line' in url:
                return ""
            if(join_set):
                parse = urlparse(url)
                visited_host.add(parse.scheme + '://' + parse.netloc)
        else:
            print('"' + response.headers['Content-Type'], '" is wrong content type.')
            if(debug):
                show_debug(url,error=True)
    return text.lower()

In [45]:
import os, codecs
from bs4 import BeautifulSoup
from urllib.parse import unquote

def write_html(url,html_source):
    global WEB_TYPE
    filepath = ''
    filename = 'dummy'
    result = urlparse(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    if(result.path.endswith(WEB_TYPE)):
        filepath = 'food/' + unquote(result.netloc + result.path[:result.path.rfind('/')])
        filename = result.path[result.path.rfind('/')+1:]
    else: 
        filepath = 'food/' + unquote(result.netloc + result.path)

    os.makedirs(filepath, 0o755, exist_ok=True)
    abs_path = filepath + '/' + filename
    if(debug):
        print('write on file directory : ',abs_path)
    f = codecs.open(abs_path ,'w' ,'utf-8' )
    f.write(soup.get_text())
    f.close()

In [46]:
# parser link from html.

def absolute_link(url):
    o = urlparse(url)
    base_url = o.scheme + '://' + o.netloc
    return  base_url + o.path

def link_parser(url):
    html_source = get_page(url)
    if(html_source == ''): return []
    write_html(url,html_source)
    if(debug):
        show_debug(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    all_el = soup.find_all('a',href=True)
    return [absolute_link(urljoin(url,el['href'].strip())) for el in all_el]

In [47]:
# write html > 10000 page.

while PAGE_COUNT <= 10000 and frontier_q:
    # backup url that was pending in queue.
    try:
        if PAGE_COUNT%100 == 0 :
            f = codecs.open('pending_queue2.txt' ,'w' ,'utf-8' )
            f.write(str(PAGE_COUNT) + '\n'+ '\n'.join(frontier_q))
            f.close()
        current_url = dequeue()
        visited_q.append(current_url)
        enqueue(link_parser(current_url))
    except Exception as err:
        print(err)
        pass

write on file directory :  food/www.7eleven.co.th//dummy
page:  5001
write on file directory :  food/sou-dai.com//dummy
page:  5002
write on file directory :  food/numsups.com//dummy
page:  5003
write on file directory :  food/www.7eleven.co.th/promotion/dummy
page:  5004
write on file directory :  food/www.7eleven.co.th/new-product/dummy
page:  5005
write on file directory :  food/www.7eleven.co.th/allmember/non-member/dummy
page:  5006
write on file directory :  food/www.7eleven.co.th/find-store/dummy
page:  5007
write on file directory :  food/www.7eleven.co.th/coupon/dummy
page:  5008
write on file directory :  food/www.7eleven.co.th/promotion/only/2433-ttb/dummy
page:  5009
write on file directory :  food/www.7eleven.co.th/promotion/tmw/2596-เปิดซิมทรูที่เซเว่น-อิ่มคุ้มฟรีทั้งปี/dummy
page:  5010
write on file directory :  food/www.7eleven.co.th/campaign-privilege/campaign/2590-winner/dummy
page:  5011
write on file directory :  food/www.7eleven.co.th/promotion/tmw/2227-tmw/dummy


write on file directory :  food/sou-dai.com/tag/ต้มยำ//dummy
page:  5085
write on file directory :  food/sou-dai.com/tag/ทำง่ายอร่อย//dummy
page:  5086
write on file directory :  food/sou-dai.com/tag/น้ำปลา//dummy
page:  5087
write on file directory :  food/sou-dai.com/tag/น้ำพริก//dummy
page:  5088
write on file directory :  food/sou-dai.com/tag/น้ำพริกปลากระป๋อง//dummy
page:  5089
write on file directory :  food/sou-dai.com/tag/น้ำพริกหนุ่ม//dummy
page:  5090
write on file directory :  food/sou-dai.com/tag/น้ำพริกอ่อง//dummy
page:  5091
write on file directory :  food/sou-dai.com/tag/บิบิมบับ//dummy
page:  5092
write on file directory :  food/sou-dai.com/tag/ปลากระป๋อง//dummy
page:  5093
write on file directory :  food/sou-dai.com/tag/ปลาเค็ม//dummy
page:  5094
write on file directory :  food/sou-dai.com/tag/ผลไม้//dummy
page:  5095
write on file directory :  food/sou-dai.com/tag/ผัก//dummy
page:  5096
write on file directory :  food/sou-dai.com/tag/มาม่า//dummy
page:  5097
write on 

write on file directory :  food/www.7eleven.co.th/new-product/bakery/248-กล้วยไข่มะพร้าวอ่อน/dummy
page:  5177
write on file directory :  food/www.7eleven.co.th/new-product/bakery/247-ต้มข่าไก่/dummy
page:  5178
write on file directory :  food/www.7eleven.co.th/new-product/bakery/246-น้ำพริกอ่อง/dummy
page:  5179
write on file directory :  food/www.7eleven.co.th/new-product/bakery/245-แกงเหลืองปลายอดมะพร้าว/dummy
page:  5180
write on file directory :  food/www.7eleven.co.th/new-product/bakery/244-หมูคั่วกลิ้ง/dummy
page:  5181
write on file directory :  food/www.7eleven.co.th/new-product/bakery/242-ต้มแซ่บหมู/dummy
page:  5182
write on file directory :  food/www.7eleven.co.th/new-product/bakery/241-ปีกไก่ไข่พะโล้/dummy
page:  5183
write on file directory :  food/www.7eleven.co.th/new-product/bakery/243-แกงเขียวหวานลูกชิ้นปลา/dummy
page:  5184
write on file directory :  food/www.7eleven.co.th/new-product/bakery/240-ปลาผัดขึ้นฉ่าย/dummy
page:  5185
write on file directory :  food/www.7el

HTTP error occurred: 500 Server Error: Internal Server Error for url: https://www.7eleven.co.th/campaign-privilege/game-activity/tel:028267777
write on file directory :  food/www.7eleven.co.th/service/health/2580-sangktan/dummy
page:  5230
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/2076-clams/dummy
page:  5231
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/2035-fish-tofu/dummy
page:  5232
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/1989-korean-noodles/dummy
page:  5233
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/1941-roasted-pork-belly-with-salt/dummy
page:  5234
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/1920-banana-pancake-choco-fruit/dummy
page:  5235
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/1897-bear/dummy
page:  5236
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/1879-banana-fritter/dummy
page:  5237
write on file di

write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/921-apple-milkshake/dummy
page:  5307
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/920--salim/dummy
page:  5308
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/919-fried-egg/dummy
page:  5309
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/918--egg-noodles/dummy
page:  5310
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/917-puff/dummy
page:  5311
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/916-sushi-rolls/dummy
page:  5312
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/915-rainbow/dummy
page:  5313
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/914-fired-noodle/dummy
page:  5314
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/913-bread-roll/dummy
page:  5315
write on file directory :  food/www.7eleven.co.th/lifestyle/cooking/912-pork-sausage/dummy
page

...

write on file directory :  food/www.lovefitt.com/diet-menu/page/8//dummy
page:  9618
write on file directory :  food/www.lovefitt.com/diet-menu/page/9//dummy
page:  9619
write on file directory :  food/www.lovefitt.com/diet-menu/page/10//dummy
page:  9620
write on file directory :  food/www.lovefitt.com/diet-menu/page/11//dummy
page:  9621
write on file directory :  food/allwellhealthcare.com/chinese-new-year-food//dummy
page:  9622
write on file directory :  food/allwellhealthcare.com/dash-diet//dummy
page:  9623
write on file directory :  food/allwellhealthcare.com/ketogenic-diet//dummy
page:  9624
write on file directory :  food/allwellhealthcare.com/how-to-use-thermometer//dummy
page:  9625
write on file directory :  food/allwellhealthcare.com/check-food-and-drugs//dummy
page:  9626
write on file directory :  food/allwellhealthcare.com/food-for-elderly//dummy
page:  9627
write on file directory :  food/allwellhealthcare.com/elderly-sleepless//dummy
page:  9628
write on file directo

write on file directory :  food/program.thaipbs.or.th/spiritofasia/dummy
page:  9697
write on file directory :  food/program.thaipbs.or.th/homklinsiam/dummy
page:  9698
write on file directory :  food/program.thaipbs.or.th/ricegrain/dummy
page:  9699
write on file directory :  food/program.thaipbs.or.th/programs/archived/dummy
page:  9700
write on file directory :  food/program.thaipbs.or.th/nareekrajang/dummy
page:  9701
write on file directory :  food/www.thaipbs.or.th/live/dummy
page:  9702
write on file directory :  food/program.thaipbs.or.th/nareekrajang/episodes/50589/dummy
page:  9703
write on file directory :  food/program.thaipbs.or.th/nareekrajang/episodes/50654/dummy
page:  9704
write on file directory :  food/thestandard.co/now-next-2020-food-drink-trends//dummy
page:  9705
write on file directory :  food/thestandard.co/category/pop/page/2//dummy
page:  9706
write on file directory :  food/thestandard.co/category/pop/page/20//dummy
page:  9707
Other error occurred: HTTPSCon

write on file directory :  food/www.maerakluke.com/topics/category/การตั้งครรภ์/การคลอดและหลังคลอด/page/3/dummy
page:  9744
write on file directory :  food/www.maerakluke.com/topics/category/การตั้งครรภ์/การคลอดและหลังคลอด/page/16/dummy
page:  9745
write on file directory :  food/www.maerakluke.com/topics/31292/dummy
page:  9746
write on file directory :  food/www.maerakluke.com/topics/31278/dummy
page:  9747
write on file directory :  food/www.maerakluke.com/topics/31272/dummy
page:  9748
write on file directory :  food/www.maerakluke.com/topics/31249/dummy
page:  9749
write on file directory :  food/www.maerakluke.com/topics/31158/dummy
page:  9750
write on file directory :  food/www.maerakluke.com/topics/27915/dummy
page:  9751
write on file directory :  food/www.maerakluke.com/topics/30535/dummy
page:  9752
write on file directory :  food/www.maerakluke.com/topics/30528/dummy
page:  9753
write on file directory :  food/www.maerakluke.com/topics/30478/dummy
page:  9754
write on file

write on file directory :  food/www.maerakluke.com/topics/28334/dummy
page:  9836
write on file directory :  food/www.maerakluke.com/topics/26661/dummy
page:  9837
write on file directory :  food/www.maerakluke.com/topics/26640/dummy
page:  9838
write on file directory :  food/www.maerakluke.com/topics/26204/dummy
page:  9839
write on file directory :  food/www.maerakluke.com/topics/23757/dummy
page:  9840
write on file directory :  food/www.maerakluke.com/topics/23790/dummy
page:  9841
write on file directory :  food/www.maerakluke.com/topics/25877/dummy
page:  9842
write on file directory :  food/www.maerakluke.com/topics/23014/dummy
page:  9843
write on file directory :  food/www.maerakluke.com/topics/category/การเลี้ยงลูก/การเลี้ยงลูกด้วยนมแม่/page/2/dummy
page:  9844
write on file directory :  food/www.maerakluke.com/topics/category/การเลี้ยงลูก/การเลี้ยงลูกด้วยนมแม่/page/3/dummy
page:  9845
write on file directory :  food/www.maerakluke.com/topics/category/การเลี้ยงลูก/การเลี้ยงล

write on file directory :  food/www.maerakluke.com/topics/1883/dummy
page:  9930
write on file directory :  food/www.maerakluke.com/topics/1748/dummy
page:  9931
write on file directory :  food/www.maerakluke.com/topics/1428/dummy
page:  9932
write on file directory :  food/www.maerakluke.com/topics/1286/dummy
page:  9933
write on file directory :  food/www.maerakluke.com/topics/609/dummy
page:  9934
write on file directory :  food/www.maerakluke.com/topics/425/dummy
page:  9935
write on file directory :  food/www.maerakluke.com/topics/398/dummy
page:  9936
write on file directory :  food/www.maerakluke.com/topics/category/พัฒนาการเด็ก/พัฒนาการเด็ก1-3ขวบ/page/2/dummy
page:  9937
write on file directory :  food/www.maerakluke.com/topics/7847/dummy
page:  9938
write on file directory :  food/www.maerakluke.com/topics/5710/dummy
page:  9939
write on file directory :  food/www.maerakluke.com/topics/2149/dummy
page:  9940
write on file directory :  food/www.maerakluke.com/topics/10006/dummy

In [48]:
# write all host in file after run 10000 page successfully.
if(debug):
  print(visited_host)
  f = codecs.open('list_host.txt' ,'w' ,'utf-8' )
  f.write('\n'.join([host for host in visited_host]))
  f.close()

{'http://program.thaipbs.or.th', 'https://brilhodealuguel.com', 'http://webboard.health.sanook.com', 'http://www.kapook.com', 'https://promotion.sanook.com', 'https://food.mthai.com', 'https://baby.kapook.com', 'https://cooking.kapook.com', 'https://www.sanook.com', 'http://www.oknation.net', 'http://women.sanook.com', 'https://www.wongnai.com', 'http://www.foodtravel.tv', 'https://seeme.me', 'https://www.bdmswellness.com', 'https://cartoon.mthai.com', 'http://www.thaifoodmaster.com', 'https://women.mthai.com', 'https://mthai.com', 'https://sivasatciftligi.com', 'https://women.kapook.com', 'https://www.lovefitt.com', 'https://bowwyyuiyee.wordpress.com', 'http://cooking.kapook.com', 'http://www.lisaguru.com', 'https://medthai.com', 'https://travel.sanook.com', 'https://travel.kapook.com', 'https://th.wikipedia.org', 'https://adaymagazine.com', 'https://www.bumrungrad.com', 'https://www.mthai.com', 'https://guru.sanook.com', 'https://th.wikibooks.org', 'https://horoscope.mthai.com', 'htt

In [ ]:
# find robots.txt in all host and write list of host that have robots.txt or sidemap.
# robots_host = []
# sitemap_host = []
# for host in visited_host:
#     text = get_page(host + '/robots.txt', 'text/plain',False)
#     if(text):
#         robots_host.append(urlparse(host).netloc)
#         if('sitemap:' in text):
#             sitemap_host.append(urlparse(host).netloc)

# f = codecs.open( 'list_robots.txt' , 'w', 'utf-8') 
# f.write('\n'.join(robots_host))
# f.close()

# f = codecs.open('list_sitemap.txt', 'w' , 'utf-8')
# f.write('\n'.join(sitemap_host))
# f.close()